<a href="https://colab.research.google.com/github/tomonari-masada/course2024-nlp/blob/main/08_sentiment_analysis_with_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLMによる感情分析

## 今日の目的
* いまどのくらい手軽にLLMを使えるようになっているかを、とりあえず体感する。
* 技術的な詳細は次回以降学んでいくことにして、とにかく使ってみる。

## 今日の内容
* 今日は、WRIMEというデータセットを使って、LLMに感情分析させてみる。
* 感情分析とは、テキストが表す感情を分析するタスク。
* 今回は、ポジティブな感情か、ネガティブな感情かの2値分類タスクとして解く。
* LLMとしては`elyza/Llama-3-ELYZA-JP-8B`を使う。
  * プロンプトを使ったテキスト生成によって感情分析の問題を解く。

* **ランタイムのタイプをGPUに設定しておく。**

## インストール
* Hugging Faceの各種ライブラリを使えば、簡単なコードを書くだけでLLMを使える。

In [ ]:
!pip install -U transformers datasets bitsandbytes accelerate

## 準備

In [ ]:
!nvidia-smi

In [ ]:
from tqdm.auto import tqdm
import numpy as np
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import (
    set_seed,
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
)

set_seed(123)

## データセット
* WRIME: 主観と客観の感情分析データセット
  * https://github.com/ids-cv/wrime
* Hugging Faceのdatasets hubに登録されているので、簡単に使える。
  * https://huggingface.co/datasets/shunk031/wrime

* `ver2`の方を使う。

In [ ]:
dataset = load_dataset(
    "shunk031/wrime",
    "ver2",
    trust_remote_code=True,
)

In [ ]:
dataset

## 3値分類問題への単純化

* ラベルから感情を表すテキストへのマッピングをおこなうリストを作っておく。
* 元のデータでは、-2, -1, 0, 1, 2の順でネガティブからよりポジティブになる。
  * 0がニュートラル。
* それぞれの感情をどのような単語で表せば良いかについて、特に正解はない。

In [ ]:
label_to_text = {
    -2: "ネガティブ",
    -1: "ネガティブ",
    0: "ニュートラル",
    1: "ポジティブ",
    2: "ポジティブ",
}

## LLM


* 今回は、elyza/Llama-3-ELYZA-JP-8Bを使う。
  * https://huggingface.co/elyza/Llama-3-ELYZA-JP-8B
* Google Colab無料版では大きすぎて使えない。
* そこで、量子化して使う。

In [ ]:
model_name = "elyza/Llama-3-ELYZA-JP-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_storage=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
).eval()

In [ ]:
model

### In-context learningを試す

* 対義語を答えさせてみる。

In [ ]:
text = "Q:高い\nA:低い\n\nQ:大きい\nA:小さい\n\nQ:狭い\nA:広い\n\nQ:少ない\nA:多い\n\nQ:速い\nA:遅い\n\nQ:嬉しい\nA:"
print(text)

In [ ]:
token_ids = tokenizer.encode(text, return_tensors="pt")
print(token_ids)

In [ ]:
with torch.no_grad():
  output_ids = model.generate(
      input_ids=token_ids.to(model.device),
      max_new_tokens=10,
  )
print(output_ids)

In [ ]:
output = tokenizer.decode(
    output_ids.tolist()[0][token_ids.size(1):], skip_special_tokens=True
)
print(output)

## プロンプト作成用のヘルパ関数

In [ ]:
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。特に指示が無い場合は、常に日本語で回答してください。"

MY_PROMPT = "次の文章はどのような感情を表していますか。¥nポジティブ、ネガティブ、ニュートラルのいずれかで答えてください。¥n"

def make_input_sequence(text):
  messages = [
      {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
      {"role": "user", "content": MY_PROMPT + text + "¥n答え:"},
  ]
  prompt = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )
  token_ids = tokenizer.encode(
      prompt, add_special_tokens=False, return_tensors="pt"
  )
  return token_ids

In [ ]:
token_ids = make_input_sequence(dataset["train"]["sentence"][0])
token_ids

In [ ]:
print(tokenizer.decode(token_ids[0]))

## 感情分析を行うヘルパ関数

In [ ]:
def sentiment_analysis(text):
  token_ids = make_input_sequence(text)
  with torch.no_grad():
    output_ids = model.generate(
        input_ids=token_ids.to(model.device),
        max_new_tokens=10,
    )
  return tokenizer.decode(
      output_ids.tolist()[0][token_ids.size(1):], skip_special_tokens=True
  )

In [ ]:
sentiment_analysis(dataset["train"]["sentence"][0])

In [ ]:
print(label_to_text[dataset["train"][0]["avg_readers"]["sentiment"]])

### 感情分析の実行

In [ ]:
for _ in range(10):
  i = np.random.randint(len(dataset["train"]))
  print(f'[{i}]' + '-'*80)
  text = dataset["train"]["sentence"][i]
  print(f"text:{text}")
  prediction = sentiment_analysis(text)
  ground_truth = dataset["train"][i]["avg_readers"]["sentiment"]
  print(f"prediction:{prediction}")
  print(f"ground truth:{label_to_text[ground_truth]}")
  print('-'*80)

# 課題
* LLMにもっとうまく感情分析をさせるプロンプトを考えてみよう。